In [5]:
! pip install PyMuPDF

Defaulting to user installation because normal site-packages is not writeable


In [6]:
import fitz  # PyMuPDF
import requests
import os, json
pdf_url = "https://people.eecs.berkeley.edu/~vazirani/algorithms/chap2.pdf"
pdf_filename = "chap2.pdf"
response = requests.get(pdf_url)
if response.status_code == 200:
    with open(pdf_filename, 'wb') as pdf_file:
        pdf_file.write(response.content)
else:
    print("Failed to download the PDF file")
    exit()

def extract_text_from_pdf(pdf_path):
    text = ""
    with fitz.open(pdf_path) as doc:
        for page in doc:
            text += page.get_text()
    return text

pdf_text = extract_text_from_pdf(pdf_filename)

In [7]:
pdf_json = {
    "content": pdf_text
}


In [8]:
json_filename = "chap2.json"
with open(json_filename, 'w') as json_file:
    json.dump(pdf_json, json_file, indent=4)


In [9]:
os.remove(pdf_filename)

print(f"PDF content has been converted to JSON and saved to {json_filename}")


PDF content has been converted to JSON and saved to chap2.json


In [10]:
with open('chap2.json', 'r') as file:
    data = json.load(file)

In [11]:
def split_text(text, max_length=3000):
    sentences = text.split('.')
    chunks = []
    chunk = ""
    for sentence in sentences:
        if len(chunk) + len(sentence) <= max_length:
            chunk += sentence + '.'
        else:
            chunks.append(chunk)
            chunk = sentence + '.'
    chunks.append(chunk)  # add the last chunk
    return chunks

chunks = split_text(data['content'])


In [12]:
! pip install torch

Defaulting to user installation because normal site-packages is not writeable


In [13]:
! pip install accelerate

Defaulting to user installation because normal site-packages is not writeable


In [14]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2-7B-Instruct",
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-7B-Instruct")


/home/rten/.local/lib/python3.10/site-packages/accelerate/utils/modeling.py:1381: UserWarning: Current model requires 469765632 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 4/4 [00:00<00:00, 11.54it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [15]:
def get_answer_from_chunk(chunk, question):
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": question},
        {"role": "user", "content": chunk}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(device)

    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=512
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return response

# Example question
question = "What is the main topic of the PDF?"

# Iterate through chunks to get answers
for chunk in chunks:
    answer = get_answer_from_chunk(chunk, question)
    print(answer)
#final_answer = get_answer_from_chunk(answer, question)




The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
/home/rten/.local/lib/python3.10/site-packages/transformers/generation/utils.py:1797: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cuda, whereas the model is on cpu. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cpu') before running `.generate()`.
  warnings.warn(


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument index in method wrapper_CUDA__index_select)